### Import necessary modules

In [1]:
import os
import csv
import numpy as np
import scipy
from scipy import integrate
import time

### Define constants

In [2]:
ATOMS = 37
ANGSTROM = 1.88972612 # the value of one angstrom in Hartree atomic units
AMU = 1822.8885 # value of 1 amu in Hartree atomic units
H_MASS = 1.00784 * AMU
C_MASS = 12.0096 * AMU
O_MASS = 15.99903 * AMU
elements = ("H", "C", "O")

### Define functions to get atomic features

In [3]:
def atomic_number(element):
    if element == "H":
        protons = 1
    if element == "C":
        protons = 6
    if element == "O":
        protons = 8
    return protons

def atomic_weight(element):
    if element == "H":
        weight = H_MASS
    if element == "C":
        weight = C_MASS
    if element == "O":
        weight = O_MASS
    return weight

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

### Import atomic coordinates of fluorescein molecule

In [4]:
nuclear_angstroms = np.empty((ATOMS,3))
nuclear_elements = np.empty(ATOMS, dtype=str)
os.chdir("/Users/tylerdougan/Dropbox (HMS)/Documents/Classes/Quantum Mechanics/Final Project/")
with open("fluorescein.csv", newline="") as csvfile:
    csv_file_reader = csv.reader(csvfile)
    for atom, row in enumerate(csv_file_reader):
        nuclear_angstroms[atom, 0:2] = row[0:2]
        nuclear_elements[atom] = row[3]

nuclear_coordinates = nuclear_angstroms * ANGSTROM
nuclear_charges = np.empty(ATOMS)
atomic_masses = np.empty(ATOMS)
for atom, element in enumerate(nuclear_elements):
    nuclear_charges[atom] = atomic_number(element)
    atomic_masses[atom] = atomic_weight(element)

### Define basis functions

In [5]:
def inner_product(psi1, psi2):
    integrand = lambda x,y,z: psi1(x,y,z) * psi2(x,y,z)
    integral = scipy.integrate.nquad(
        integrand,
        ranges=[[-np.inf,np.inf], [-np.inf,np.inf], [-np.inf,np.inf]])
    return integral

# import basis set
basis_started = False
basis_functions = dict([(element, dict()) for element in elements])
with open("sto-2g.bse") as csvfile:
    csv_file_reader = csv.reader(csvfile)
    for atom, row in enumerate(csv_file_reader):
        if len(row) > 0:
            line = str.split(row[0])
            if line[0] == "Element:":
                current_element = line[1]
            if line[0] == "Shell:":
                current_shell = int(line[2]) + 1
                current_orbital = line[-1]
                for degeneracy in current_orbital:
                    basis_functions[current_element].update(
                        {(current_shell, degeneracy) : []})
            if isfloat(line[0]):
                for orbit, degeneracy in enumerate(current_orbital):
                    basis_functions[current_element][(current_shell, degeneracy)].append(
                        (float(line[0]), float(line[orbit+1])))

exponents = []
for atom, element in enumerate(nuclear_elements):
    exponents.append([])
    for shell in basis_functions[element]:
        for alpha in basis_functions[element][shell]:
            if float(alpha[0]) not in exponents[atom]:
                exponents[atom].append(float(alpha[0]))

### Integrate
scipy.integrate.nquad uses FORTRAN QUADPAC algorithm

In [6]:
t1 = time.time()
h1s = lambda x,y,z: basis_functions["H"][(1, "S")][0][1] * (
    2*basis_functions["H"][(1, "S")][0][0] / np.pi)**(3/2) * np.exp(
    -basis_functions["H"][(1, "S")][0][0] * (x**2 + y**2 + z**2)) + basis_functions["H"][(1, "S")][1][1] * (
    2*basis_functions["H"][(1, "S")][1][0] / np.pi)**(3/2) * np.exp(
    -basis_functions["H"][(1, "S")][1][0] * (x**2 + y**2 + z**2))
psi1psi2 = inner_product(h1s, h1s)
t2 = time.time()
print(t2 - t1)

95.44831109046936
